In [161]:
def importacion[P, Q](q: Q): P => Q =
  (_: P) => q

def exportacion[P, Q](sipeq: P => Q): P => Q =
  (p: P) => sipeq(p)

def conmutatividad_conjuncion [P, Q](p: P, q: Q): (Q, P) =
  (q, p)

def conmutatividad_disyuncion[P, Q](e: Either[P, Q]): Either[Q, P] =
  e match {
    case Left(p) => Right(p)
    case Right(q) => Left(q)
  }

def distribucion[P, Q, R](e: Either[P, (Q, R)]): (Either[P, Q], Either[P, R]) =
  e match {
    case Left(p) => (Left(p), Left(p))
    case Right((q, r)) => (Right(q), Right(r))
  }


def alternativa[P, Q](p1: Either[P, Q], p2: Not[P]): Q = {
  p1 match {
    case Left(p) => p2(p)
    case Right(q) => q 
  }
}

def rtr[P,Q,R] (p1: P=>Q, p2: Q=>R): P=>R =
    p=> p2(p1(p))

def modusTollens[P, Q](pToQ: P => Q)(notQ: Not[Q]): Not[P] =
    (p: P) => notQ(pToQ(p))

def rImp[P,Q,R](p1: P=>(Q=>R)):(P,Q)=>R=
    (p,q)=>p1(p)(q)

def rExp[P,Q,R](p1: ((P,Q))=>R): P=>(Q=>R)=
    p=>(q=> p1((p,q)))

def rcontr1[P,Q](p1:P=>Q): Not[Q] => Not[P]=
    nq=> modusTollens(p1)(nq)

def rcontr1_1[P,Q](DN: DoubleNegation)(p1:Not[P]=>Q): Not[Q] => P=
    nq=> DN(modusTollens(p1)(nq))

def rcontr1_2[P,Q](p1:P=>Not[Q]): Q => Not[P]=
    q=> { val nnq= DN2(q)
        modusTollens(p1)(nnq)}

def rcontr2[P, Q](DN: DoubleNegation)(p1: Not[Q] => Not[P]): P => Q = 
    p => DN.apply[Q](nq => p1(nq)(p)) 

def ria1[P,Q](p1: Either[P,Q], p2:Not[P]):Q=
    p1 match{
        case Left(p) => p2(p)
        case Right(q)=> q

    }

def ria2[P,Q](p1: Either[P,Q], p2:Not[Q]):P=
    p1 match{
        case Left(p)=>p
        case Right(q) => p2(q)
    }

def rDf1[P,Q](p1: Not[Either[P,Q]]):(Not[P], Not[Q])={
    val np: Not[P] = p=> p1(Left(p))
    val nq: Not[Q] = q=> p1(Right(q))
    (np,nq)
}

def rDf1_1[P,Q](EM: ExcludedMiddle)(p1: Not[Either[Not[P],Not[Q]]]):(P, Q)={
    val p = EM[P] match {
                    case Left(p) => p
                    case Right(notP) => 
                    val enp: Either[Not[P], Not[Q]] = Left(notP)
                    p1(enp)
              }
    val q = EM[Q] match {
                    case Left(q) => q
                    case Right(notQ) => 
                    val enq: Either[Not[P], Not[Q]] = Right(notQ)
                    p1(enq)
              }
  (p, q)
}


def rDf1_2[P,Q](EM: ExcludedMiddle)(p1: Not[Either[Not[P],Q]]):(P, Not[Q])={
    val p: P = EM[P] match {
    case Left(p) => p
    case Right(np) => 
      val either = Left(np)
      p1(either)
    }

    val nq: Not[Q] = q => {
    val either = Right(q)
        p1(either)
    }
  (p, nq)
}


def rDf1_And[P, Q](EM: ExcludedMiddle)(p1:Not[(P, Q)]) : Either[Not[P], Not[Q]] = { 
    EM[P] match {
        case Left(p) => Right((q) => p1((p, q)))
        case Right(np) => Left((p => np(p)))
    }
}

def rDf2[P,Q](p1:(Not[P], Not[Q])): Not[Either[P,Q]]=
    e=> e match{
        case Left(p)=> p1._1(p)
        case Right(q)=> p1._2(q)
        }

def rDf3_v2[P,Q] (p1:Either[P,Q]): Not[P]=>Q=
    np=> p1 match{
        case Left(p)=>np(p)
        case Right(q)=>q
}

def rDfAnd[P,Q] (p1:(P,Q)): Not[P=>Not[Q]]=
    aux=> aux(p1._1)(p1._2)

def rDfAnd2[P,Q] (p1:(P,Q)): Not[Either[Not[P],Not[Q]]]=
    e => e match{
        case Left(np) => np(p1._1)
        case Right(nq) => nq(p1._2)
    }

def rDfEither[P,Q](p1: Either[P,Q]): Not[(Not[P],Not[Q])]=
    t => p1 match{
        case Left(p) => t._1(p)
        case Right(q) => t._2(q)
    }

def rdfNotAnd [P,Q](EM: ExcludedMiddle)(p1:Not[(P,Q)]): Either[Not[P],Not[Q]]={
   EM[P] match {
    case Left(p) =>
      Right((q: Q) => p1((p, q)))
    case Right(np) =>
      Left(np)
  }
}

def rdfNotAndImp[P,Q] (EM: ExcludedMiddle)(p1:Not[(P,Not[Q])]): P=>Q = {
  (p: P) => EM[Q] match {
      case Left(q) => q
      case Right(nq) => val np: Not[P] = (p: P) => p1((p, nq))
                        np(p)
  }
}
def rdfNotAndImp2 [P,Q](EM: ExcludedMiddle)(p1:Not[Not[P]=>Q]):(Not[P],Not[Q])={
  EM[P] match {
    case Left(p) =>
      val nq: Not[Q] = (q:Q) => p1((np: Not[P]) => np(p))
      val np: Not[P] = (p:P) => p1((np: Not[P]) => np(p))
      (np, nq)
    case Right(np) =>
      val nq: Not[Q] = (q: Q) => p1((np: Not[P]) => q)
      (np, nq)
  }
}

def rDfImp[P,Q](p1: P=>Q): Not[(P,Not[Q])]=
    t=> {val q = p1(t._1)
        t._2(q)}

def reCoimp1[P,Q](p1:P<=>Q, p2:P):Q=
    p1._1(p2)

def reCoimp2[P,Q](p1:P<=>Q, p2:Not[Q]):Not[P]=
    modusTollens(p1._1)(p2)
    
def reCoimp3[P,Q](p1:P<=>Q, p2:Q):P=
    p1._2(p2)

def reCoimp4[P,Q](p1:P<=>Q, p2:Not[P]):Not[Q]=
    modusTollens(p1._2)(p2)

trait DoubleNegation {
  def apply[P](nnp: Not[Not[P]]): P
}

trait ExcludedMiddle { 
    def apply[P]: Either[P, Not[P]]
}

def DN2[P](p: P): Not[Not[P]] = 
                //P=>Nothing=>Nothing
     np=> np(p)

type Not[P] = P => Nothing
type <=>[P, Q] = (P => Q, Q => P)

defined function importacion
defined function exportacion
defined function conmutatividad_conjuncion
defined function conmutatividad_disyuncion
defined function distribucion
defined function alternativa
defined function rtr
defined function modusTollens
defined function rImp
defined function rExp
defined function rcontr1
defined function rcontr1_1
defined function rcontr1_2
defined function rcontr2
defined function ria1
defined function ria2
defined function rDf1
defined function rDf1_1
defined function rDf1_2
defined function rDf1_And
defined function rDf2
defined function rDf3_v2
defined function rDfAnd
defined function rDfAnd2
defined function rDfEither
defined function rdfNotAnd
defined function rdfNotAndImp
defined function rdfNotAndImp2
defined function rDfImp
defined function reCoimp1
defined function reCoimp2
defined function reCoimp3
defined function reCoimp4
defined trait DoubleNegation
defined trait ExcludedMiddle
defined function DN2
defined type Not
defined type <=>

In [3]:
def example [P,Q,R,S] (p1: Either[Not[P],Not[Q]], p2:R=>P, p3: S=>Q): Either [Not[R],Not[S]]=
    p1 match {
        case Left (np) => Left((r:R)=> np(p2(r)))
        case Right (nq) => Right ((s:S) =>nq (p3(s)))
}

defined function example

In [4]:
def proof[P, Q, R, S](
        p1: Not[P] Either Not[Q], 
        p2: R => P, 
        p3: S => Q): Not[R] Either Not[S] = 

    p1 match{                                // Ev 
        case Left(np) => 
            val nr: Not[R] = modusTollens(p2)(np)  // Tollens
            Left(nr)                          // Iv
        case Right(nq) => 
            val ns: Not[S] = modusTollens(p3)(nq)  // Tollens
            Right(ns)   }                      // Iv

defined function proof

In [5]:
def importacion[P, Q](q: Q): P => Q =
  (_: P) => q

defined function importacion

In [6]:
def exportacion[P, Q](sipeq: P => Q): P => Q =
  (p: P) => sipeq(p)


defined function exportacion

In [7]:
def conmutatividad_conjuncion [P, Q](p: P, q: Q): (Q, P) =
  (q, p)


defined function conmutatividad_conjuncion

In [8]:
def conmutatividad_disyuncion[P, Q](e: Either[P, Q]): Either[Q, P] =
  e match {
    case Left(p) => Right(p)
    case Right(q) => Left(q)
  }


defined function conmutatividad_disyuncion

In [9]:
def distribucion[P, Q, R](e: Either[P, (Q, R)]): (Either[P, Q], Either[P, R]) =
  e match {
    case Left(p) => (Left(p), Left(p))
    case Right((q, r)) => (Right(q), Right(r))
  }


defined function distribucion

In [10]:
def ex1[P, Q, R](p1: ((P, Q)) => R, p2: P): Q => R = {
  q: Q => p1((p2, q))
}



defined function ex1

In [183]:
//sin reglas 
def ex2[P, Q, R, S](p1: P => Q, p2: Either[R, S], p3: S => Not[Q], p4: Not[R]): Not[P] = {
  p: P => {
    p2 match {
      case Left(r) => p4(r)
      case Right(s) => 
        val q: Q = p1(p)
        p3(s)(q)
    }
  }
}

defined function ex2

In [182]:
//con reglas 
def ex2[P, Q, R, S](p1: P => Q, p2: Either[R, S], p3: S => Not[Q], p4: Not[R]): Not[P] = {
    val s = ria1(p2,p4)
    val nq = p3(s)
    modusTollens(p1)(nq)
}

defined function ex2

In [12]:
trait Not[P] {
  def apply(p: P): Nothing
}

type ¬[P] = Not[P]

def modusTollens[P, Q](pToQ: P => Q)(notQ: Not[Q]): Not[P] =
  (p: P) => notQ(pToQ(p))


defined trait Not
defined type ¬
defined function modusTollens

In [21]:
def modusTollens[P, Q](pToQ: P => Q)(notQ: Not[Q]): Not[P] =
  (p: P) => notQ(pToQ(p))


defined function modusTollens

In [19]:
type Not[P] = P => Nothing
type <=>[P, Q] = (P => Q, Q => P)

defined type Not
defined type <=>

In [20]:
type Not[P] = P => Nothing

def alternativa[P, Q](p1: Either[P, Q], p2: Not[P]): Q = {
  p1 match {
    case Left(p) => p2(p)
    case Right(q) => q 
  }
}


defined type Not
defined function alternativa

In [22]:
def ex5[P, Q, R, S](p1: R => S, p2: Either[P, Q], p3: (Not[P], Not[S])): (Q, Not[R]) = {
  val notP = p3._1
  val notS = p3._2
  val notR = modusTollens(p1)(notS)
  
  p2 match {
    case Left(p) => (alternativa(p2, notP), notR)
    case Right(q) => (q, notR)
  }
}

defined function ex5

In [23]:
def ex7 [P,Q,R] (p1: P=>(Q=>R),p2:Q):P=>R =
    p=> p1(p)(p2)


defined function ex7

In [24]:
def rcontr1[P,Q](p1:P=>Q): Not[Q] => Not[P]=
    nq=> modusTollens(p1)(nq)


defined function rcontr1

In [25]:
def rtr[P,Q,R] (p1: P=>Q, p2: Q=>R): P=>R =
    p=> p2(p1(p))

defined function rtr

In [26]:
def modusTollens[P, Q](pToQ: P => Q)(notQ: Not[Q]): Not[P] =
    (p: P) => notQ(pToQ(p))


defined function modusTollens

In [27]:
trait DoubleNegation {
  def apply[P](nnp: Not[Not[P]]): P
}

defined trait DoubleNegation

In [162]:
def DN2[P](p: P): Not[Not[P]] = //P=>Nothing=>Nothing
     np=> np(p)

defined function DN2

In [29]:
def rcontr2[P, Q](DN: DoubleNegation)(p1: Not[Q] => Not[P]): P => Q = 
     p => DN.apply[Q](nq => p1(nq)(p)) //DN(nq => p1(nq)(p)) 
                                      //DN.apply[Q](nq => p1(nq)(p))



defined function rcontr2

In [121]:
def rdfNotAndImp[P,Q] (EM: ExcludedMiddle)(p1:Not[(P,Not[Q])]): P=>Q = {
    (p: P) => EM[Q] match {
    case Left(q) => q
    case Right(nq) =>
      val np: Not[P] = (p: P) => p1((p, nq))
      np(p)
  }
}

defined function rdfNotAndImp

In [120]:
def ex6 [P,Q,R,S,T,U] (DN:DoubleNegation)(EM: ExcludedMiddle)(p1: P=>Q, p2: R=>P, p3: Not[R]=> Not[T], p4:Not[(S,Not[R])],  p5: Either[T,S]): Either[Q,U]=
                                                                                    //S=>R
    p5 match{
        case Left(t) => val tIr = rcontr2(DN)(p3) //tipo Q p1(p2(rcontr2(p3)(t))) 
                    val r = tIr(t)
                    val p = p2(r)
                    Left(p1(p)) //:Q
        case Right(s) =>val sIr = rdfNotAndImp(EM)(p4) 
                    val r = sIr(s)
                    val p = p2(r)
                    Left(p1(p)) //:Q        
    }

defined function ex6

In [163]:
def ex4[P,Q,R,S,T](p1:P=>Either[Q,R], p2: Not[R], p3: Q=>Either[S,T], p4:Not[Either[S,T]]): Not[P]={
    p => 
        val e = p1(p)
        val nq = modusTollens(p3)(p4) //MT=> Not[Q]
        e match {
            case Left(q) => nq(q) // nq(q): Nothing
    
            case Right(r) => p2(r) //nr(r): Nothing
        }
}

defined function ex4

In [35]:
def ex8 [P,Q,R,S] (p1: P=>Either[Q,R], p2: Either[Q,R]=> P, p3: P=>S, p4:Q):S=
    rtr(p2,p3)(Left(p4))

    

defined function ex8

In [144]:
def ex9 [P,Q] (p1: Not[Not[P]=>Not[Q]]): P=>Not[Q]= {
  p: P => q: Q => ???
}

defined function ex9

In [37]:
def ex10[P,Q,R,S](DN: DoubleNegation)(p1:(P,Q)=>R, p2:Not[Either[P,R]]=>S, p3: P=>Q): Not[S]=>R = {
    ns => 
        val epr= DN(modusTollens(p2)(ns)) //Either[P,R]
        epr match {
            case Left(p) => val q = p3(p)
                            p1(p,q)
            case Right(r) => r
        }
}


defined function ex10

In [165]:
def ej10[P, Q, R, S](DN: DoubleNegation)(p1: ((P, Q)) => R, p2: Not[Either[P, R]] => S, p3: P => Q): Not[S] => R = 
    (p4: Not[S]) => { val p5: Either[P, R] = DN.apply[Either[P, R]](modusTollens(p2)(p4))
                    p5 match{
                        case Left(p6: P) => val p7: Q = p3(p6)
                                val p8: (P, Q) = (p6, p7)
                                p1(p8)
                                
                        case Right(p10: R) => p10
                }
               }

defined function ej10

In [190]:
def ex11 [P,Q] (p1: Either[P,Q]): Not[P] =>Q =
    rDf3_v2(p1)


defined function ex11

In [189]:
def ex11 [P,Q] (p1: Either[P,Q]): Not[P] =>Q =
    np=> p1 match{
            case Left(p)=>np(p)
            case Right(q)=>q
        }


defined function ex11

In [43]:
def ex12[P,S,T,W](DN:DoubleNegation)(p1: S<=>T, p2: Either[T,P], p3: S=>Not[W], p4:W): P ={
    val wIns = rcontr1_2(p3)
    val ns = wIns(p4)
    val tIs = p1._2
    val nt = modusTollens(tIs)(ns)
    ria1(p2,nt) //:P
    
}


defined function ex12

In [167]:
def ex14 [P,Q,R,S,T] (p1: (P,Q)=>R, p2: (R,S)=>T):((P,Q),S)=>T={
    case((p,q),s)=> val r = p1(p,q) 
                    p2(r,s)
}

defined function ex14

In [168]:
def ex15[P,Q,R](p1: P=>(Q=>R)):Q=>(P=>R)=
    q=>p=>p1(p)(q)

defined function ex15

In [150]:
def ex17[P,Q,R](DN:DoubleNegation)(EM: ExcludedMiddle)(p1: ((P,Q))=>R):((P,Not[R])) => Not[Q] =
    t => {val nr = t._2
        val n_pAq = modusTollens(p1)(nr) //¬(p,q)
        val e_npnq= rDf1_And(EM)(n_pAq) //falta este metodo 
        val p = t._1
        val nnp = DN2(p)
        ria1(e_npnq,nnp)
    }

defined function ex17

In [47]:
def ex18[P,Q](p1: P=>(P=>Q)):P=>Q=
    p=> p1(p)(p)

defined function ex18

In [149]:
def ex19[P](EM: ExcludedMiddle)(p1:Not[P]=>P):P=
    EM[P] match {
        case Left(p) => p
        case Right(np) => p1(np)
    }

defined function ex19

In [184]:
//nota11 solucion, con gelas auxiliares
def ex20[P,Q,R,S,T](p1: P=>Not[Q], p2:Not[Q]=>(Not[R],Not[S]), p3:Either[R,S]=>T, p4:T=>Q): P=>(Not[R],Not[S])=
    rtr(p1,p2)

//con reglas auxiliares 
def ex20_v2[P,Q,R,S,T](p1: P=>Not[Q], p2:Not[Q]=>(Not[R],Not[S]), p3:Either[R,S]=>T, p4:T=>Q): P=>(Not[R],Not[S])=
    p=> {val nq = p1(p)
         val p5 = p2(nq)
          p5
        }

defined function ex20
defined function ex20_v2

In [185]:
def ria1_1[P,Q](p1: Either[Not[P],Q], p2:P):Q=
    p1 match{
        case Left(np) => np(p2)
        case Right(q)=> q
}

defined function ria1_1

In [108]:
def ex21 [P,Q,R,S,T] (DN:DoubleNegation)(p1: R=>P, p2: Not[Q]=>Not[R], p3: S=>Q, p4: ((P,Q))=>T, p5: Either[Not[S],P]):Either[R,S]=>T=
    e => e match{
        case Left(r) => val p = p1(r)
                        val rIq = rcontr2(DN)(p2)
                        val q = rIq(r)
                        val paq = ((p,q))
                        p4(paq)
                    
        case Right(s)=> val q = p3(s)
                        val p = ria1_1(p5,s) 
                        val paq = ((p,q))
                        p4(paq)
}



defined function ex21

In [123]:
//no es demostrable en la logica intuicionista, es logica clasica 


In [111]:
def rDf1_And[P, Q](EM: ExcludedMiddle)(p1:Not[(P, Q)]) : Either[Not[P], Not[Q]] = { 
    EM[P] match {
        case Left(p) => Right((q) => p1((p, q)))
        case Right(np) => Left((p => np(p)))
    }
}

defined function rDf1_And

In [169]:
def rDf2[P,Q](p1:(Not[P], Not[Q])): Not[Either[P,Q]]=
    e=> e match{
        case Left(p)=> p1._1(p)
        case Right(q)=> p1._2(q)
        }


defined function rDf2

In [55]:
def ex22[P,Q,R,S,T](p1:Either[Q,Not[S]]=>T, p2: Not[Q]=>R, p3:P=>Not[S], p4: T=>S): P=>R={
(p:P)=> 
    val ns: Not[S] = p3(p)
    val nt= modusTollens(p4)(ns)
    val ne= modusTollens(p1)(nt)
    val aux= rDf1(ne)
    p2(aux._1)
}

defined function ex22

In [155]:

def rDf1_1[P,Q](EM: ExcludedMiddle)(p1: Not[Either[Not[P],Not[Q]]]):(P, Q)={
    val p = EM[P] match {
                case Left(p) => p
                case Right(np) => 
                val enp: Either[Not[P], Not[Q]] = Left(np)
                p1(enp)
              }
    val q = EM[Q] match {
                case Left(q) => q
                case Right(nq) => 
                val enq: Either[Not[P], Not[Q]] = Right(nq)
                p1(enq)
              }
  (p, q)
}


defined function rDf1_1

In [156]:

def modusTollensNOT[P, Q](EM: ExcludedMiddle)(p1: Not[P] => Not[Q])(p2:Q):P = {
    EM[P] match {
        case Left(p) => p
        case Right(np) => 
            val nq = p1(np)
            nq(p2)
    }
}

defined function modusTollensNOT

In [158]:
def ex24[P,Q,R,S,T](EM: ExcludedMiddle)(DN:DoubleNegation)(p1: Either[P,Q]=> (R,S), p2: Not[Either[Not[P],Not[R]]], p3: Not[T]=>Not[(P,S)]):T={
    val par = rDf1_1(EM)(p2)
    val p= par._1
    val ras = p1(Left(p))
    val s = ras._2
    val pas = ((p,s))
    modusTollensNOT(EM)(p3)(pas)
}


defined function ex24

In [186]:
def ex25 [P,Q,R,S,T](p1:S=>Q):(P=>(Q=>R))=>(P=>(S=>R))=
    p2=>(p=>(s=>{
                    val q = p1(s)
                    val qir= p2(p)
                    val r = qir(q)
                    r
                }))

defined function ex25

In [187]:
def ex29[P,Q,R,S,T](p1:P=>((Q=>R)=>S)): (Q=>R) => P=>S=
    p2=> p=>{   val p4 = p1(p)
                val p5 = p4(p2)
                p5
            }

defined function ex29

In [61]:
def rcontr1_1[P,Q](DN: DoubleNegation)(p1:Not[P]=>Q): Not[Q] => P=
    nq=> DN(modusTollens(p1)(nq))

def rcontr1_2[P,Q](p1:P=>Not[Q]): Q => Not[P]=
    q=> { val nnq= DN2(q)
        modusTollens(p1)(nnq)}
 

defined function rcontr1_1
defined function rcontr1_2

In [62]:

def ex32[P,Q,R,S,T,U](DN:DoubleNegation)(p1: P<=>((Q,(T,U))), p2: Not[P], p3: R<=>Not[Q], p4:Not[R]=>T, p5: Not[U]=>R):Either[R,S]={
    val p6 = reCoimp4(p1,p2) //Not[(Q,(T,U))]
    val p7 = rdfNotAnd(p6) // Either[Not[Q],Not[(T,U)]]
    p7 match{
        case Left(nq)=> Left(p3._2(nq))//:R
        case Right(n_tAu)=> { val e = rdfNotAnd(n_tAu)
                                e match{
                                    case Left(nt)=> {val p12 = rcontr1_1(DN)(p4) //¬t=>r
                                                Left(p12(nt))}
                                    case Right(nu)=> Left(p5(nu))
                                }
                            }
    }
    
}

defined function ex32

In [63]:
def ex33[P,Q,R,S,T](p1: P=>Q, p2: Q=>R, p3: S=>T, p4:Either[S,P]): Either[R,T]=
    p4 match{
        case Left(s)=> Right(p3(s))
        case Right(p)=> Left(p2(p1(p)))
    }

defined function ex33

In [64]:
def ex36_v1[P,Q,R](p1: Either[P,Q]=>R): Either[P=>R,Q=>R]={
    val aux1 = (p:P)=> p1(Left(p))
    Left(aux1)
}


defined function ex36_v1

In [65]:
def ex36_v2[P,Q,R](p1: Either[P,Q]=>R): Either[P=>R,Q=>R]={
    val aux2 = (q:Q)=> p1(Right(q))
    Right(aux2)
}

defined function ex36_v2

In [141]:
def rdfNotAndImp2 [P,Q](EM: ExcludedMiddle)(p1:Not[Not[P]=>Q]):(Not[P],Not[Q])={
  EM[P] match {
    case Left(p) =>
      val nq: Not[Q] = (q:Q) => p1((np: Not[P]) => np(p))
      val np: Not[P] = (p:P) => p1((np: Not[P]) => np(p))
      (np, nq)
    case Right(np) =>
      val nq: Not[Q] = (q: Q) => p1((np: Not[P]) => q)
      (np, nq)
  }
}


defined function rdfNotAndImp2

In [153]:
def ex37[P,Q,R,S,T](EM: ExcludedMiddle) (p1: R=>S, p2: Either[P,Q], p3:Not[Not[P]=>S], p4: Not[P]=>Q):(Q,Not[R])={
    val p5 = rdfNotAndImp2(EM)(p3) //npAns
    val p6 = p5._1 //np
    val p7 = p4(p6) //q
    val p8 = p5._2 //ns
    val p9 = modusTollens(p1)(p8) //nr
    (p7,p9)
    
}
                                                

defined function ex37

In [69]:
def ex37[P,Q,R,S,T] (p1: R=>S, p2: Either[P,Q], p3:(Not[P],Not[S]), p4: Not[P]=>Q):(Q,Not[R])={
                                                //Not[Not[P]=>S] mirar sol hacerlo con funcion 
    val np = p3._1
    val ns = p3._2
    val q = p4(np)
    val nr= modusTollens(p1)(ns)
    (q,nr)
}


defined function ex37

In [70]:
def ex38[P,Q,R,S,T](p1:((P=>Q),R), p2: S=>T, p3:R=>S): Either[Q,T]={
    val r= p1._2
    val s= p3(r)
    val t= p2(s)
    Right(t)
}

defined function ex38

In [71]:
def rDf1[P,Q](p1: Not[Either[P,Q]]):(Not[P], Not[Q])={
    val np: Not[P] = p=> p1(Left(p))
    val nq: Not[Q] = q=> p1(Right(q))
    (np,nq)
}

defined function rDf1

In [72]:
type Not[A] = A => Nothing

trait DoubleNegation {
  def apply[P](nnp: Not[Not[P]]): P
}
def proof[P](DN: DoubleNegation): (Not[P] => P) => P =
    f => DN[P]((n: Not[P]) => n(f(n)))

defined type Not
defined trait DoubleNegation
defined function proof

In [135]:

def rDf1_2[P,Q](EM: ExcludedMiddle)(p1: Not[Either[Not[P],Q]]):(P, Not[Q])={
    val p: P = EM[P] match {
    case Left(p) => p
    case Right(np) => 
      val either = Left(np)
      p1(either)
    }

    val nq: Not[Q] = q => {
    val either = Right(q)
        p1(either)
    }
  (p, nq)
}


defined function rDf1_2

In [160]:
def ex40[P,Q,R,S,T](EM: ExcludedMiddle)(DN: DoubleNegation)(p1:Not[P]<=>Q, p2:Not[Either[Not[Q],R]], p3: Either[P,S]):Either[S,T]={
                                                        //(Q,Not[R])
    val  t = rDf1_2(EM)(p2) //(Q,Not[R])
    val q = t._1
    val np = reCoimp3(p1,q)
    val s = ria1(p3,np)
    Left(s)
}

defined function ex40

In [175]:
def ex41 [P,Q,R] (p1: P=>Q, p2:Either[R, Not[P]]):P=>(Q,R)={
    p=> val q = p1(p)
        p2 match{
        case Left(r) =>(q, r)
        case Right(np) => (q, np(p))
        }
    }

defined function ex41

In [76]:
//CON REGLAS DE INFERENCIA 
def ex41 [P,Q,R] (p1: P=>Q, p2:Either[R, Not[P]]):P=>(Q,R)={
    p=> val q = p1(p)
        val nnp = DN2(p)
        val r = ria2(p2,nnp)
        (q,r)
}

defined function ex41

In [152]:
//CON REGLAS DE INFERENCIA
def ex42[P,Q,R,S](EM: ExcludedMiddle)(p1:((P,Q))=>R, p2: R=>S, p3: (Q,Not[S]) ): Not[P]={
    val q = p3._1
    val ns = p3._2
    val paqis = rtr(p1,p2)
    val npaq = modusTollens(paqis)(ns)      
    val nponq = rDf1_And(EM)(npaq)
    val nnq =  DN2(q)
    ria2(nponq,nnq)
}


defined function ex42

In [154]:

def ex44 [P,Q,R,S,T](EM: ExcludedMiddle)(p1: ((P,Q))=>(Either[R,S]), p2: Not[(T,Not[P])], p3:T, p4: T<=>Q):Not[R]=>S ={
                                                //T=>P
    val tip= rdfNotAndImp(EM)(p2)
    val q = p4._1(p3)
    val p = tip(p3)
    val paq = (p,q)
    val e = p1(paq)
    
nr=> ria1(e,nr)
}


defined function ex44

In [80]:
//me lo he inventado sin querer
def ex44 [P,Q,R,S,T](p1: ((P,Q))=>((R,S)), p2: Either[Not[T],P], p3:T, p4: T<=>Q):Not[R]=>S ={
    val tip = p4._1
    val q = tip(p3)
    nr=> p2 match{
        case Left(nt)=>nt(p3)
        case Right(p)=> val paq = (p,q)
                        val ras = p1(paq)
                        ras._2 

    }
}


defined function ex44

In [81]:
def rImp[P,Q,R](p1: P=>(Q=>R)):(P,Q)=>R=
    (p,q)=>p1(p)(q)

defined function rImp

In [82]:
def rExp[P,Q,R](p1: ((P,Q))=>R): P=>(Q=>R)=
    p=>(q=> p1((p,q)))

defined function rExp

In [177]:
def ria1[P,Q](p1: Either[P,Q], p2:Not[P]):Q=
    p1 match{
        case Left(p) => p2(p)
        case Right(q)=> q

    }

defined function ria1

In [178]:
def ria2[P,Q](p1: Either[P,Q], p2:Not[Q]):P=
    p1 match{
        case Left(p)=>p
        case Right(q) => p2(q)
    }

defined function ria2

In [179]:
def proof[P](DN: DoubleNegation): (Not[P] => P) => P =
    f => DN[P]((n: Not[P]) => n(f(n)))

defined function proof

In [180]:
def rcontr1_1[P,Q](DN: DoubleNegation)(p1:Not[P]=>Q): Not[Q] => P=
    nq=> DN(modusTollens(p1)(nq))

def rcontr1_2[P,Q](p1:P=>Not[Q]): Q => Not[P]=
    q=> {val nnq= DN2(q)
        modusTollens(p1)(nnq)}
 

defined function rcontr1_1
defined function rcontr1_2

In [88]:
def ex45[P,Q,R,S,T] (DN:DoubleNegation)( p1: Either[P,Q], p2: R=>Not[P], p3: S=>R, p4:Not[T]=>S, p5: R<=>Not[Q]):T={
    val ntIr = rtr(p4,p3)
    p1 match{
        case Left(p) => {
                        val pInr = rcontr1_2(p2) //necesario ¬¬p=p
                        val nr = pInr(p)
                        val nrIt = rcontr1_1(DN)(ntIr)
                        nrIt(nr) }//:T
        case Right(q) => {
                         val rInq = p5._1
                         val qInr = rcontr1_2(rInq) //necesario ¬¬p=p
                         val nr = qInr(q)
                         val ntIr = rtr(p4,p3)
                         val nrIt = rcontr1_1(DN)(ntIr) //necesario ¬¬p=p
                         nrIt(nr)} //:T
    }
}


defined function ex45

In [148]:

def ria2_NOT[P, Q](DN: DoubleNegation)(EM: ExcludedMiddle)(p1: Either[Not[P], Not[Q]], p2: Q): P = {
  p1 match {
    case Left(np) => EM[P] match {
                case Left(p) => p
                case Right(np) => ???
              }
    case Right(nq) => nq(p2)
  }
}

defined function ria2_NOT

In [181]:
type Not[P] = P => Nothing
type <=>[P, Q] = (P => Q, Q => P)
def reCoimp1[P,Q](p1:P<=>Q, p2:P):Q=
    p1._1(p2)
def reCoimp2[P,Q](p1:P<=>Q, p2:Not[Q]):Not[P]=
    modusTollens(p1._1)(p2)
def reCoimp3[P,Q](p1:P<=>Q, p2:Q):P=
    p1._2(p2)
def reCoimp4[P,Q](p1:P<=>Q, p2:Not[P]):Not[Q]=
    modusTollens(p1._2)(p2)
def riCoimp[P,Q](p1: P=>Q, p2: Q=>P):P<=>Q=
(p1,p2)

defined type Not
defined type <=>
defined function reCoimp1
defined function reCoimp2
defined function reCoimp3
defined function reCoimp4
defined function riCoimp

In [139]:

def rDf3_v1[P,Q](EM: ExcludedMiddle)(p1:Not[P]=>Q ): Either[P,Q]={
    EM[P] match {
    case Left(p) => Left(p)
    case Right(np) => Right(p1(np))
  }   
}

defined function rDf3_v1

In [92]:
def rDf3_v2[P,Q] (p1:Either[P,Q]): Not[P]=>Q=
    np=> p1 match{
        case Left(p)=>np(p)
        case Right(q)=>q
    }

defined function rDf3_v2

In [93]:
def rDfAnd[P,Q] (p1:(P,Q)): Not[P=>Not[Q]]=
    aux=> aux(p1._1)(p1._2)

defined function rDfAnd

In [94]:
def rDfAnd2[P,Q] (p1:(P,Q)): Not[Either[Not[P],Not[Q]]]=
    e => e match{
        case Left(np) => np(p1._1)
        case Right(nq) => nq(p1._2)
    }


defined function rDfAnd2

In [95]:
def rDfEither[P,Q](p1: Either[P,Q]): Not[(Not[P],Not[Q])]=
    t => p1 match{
        case Left(p) => t._1(p)
        case Right(q) => t._2(q)
    }

defined function rDfEither

In [96]:
def rDfImp[P,Q](p1: P=>Q): Not[(P,Not[Q])]=
    t=> {val q = p1(t._1)
        t._2(q)}

defined function rDfImp

In [137]:
def rdfNotAnd [P,Q](EM: ExcludedMiddle)(p1:Not[(P,Q)]): Either[Not[P],Not[Q]]={
   EM[P] match {
    case Left(p) =>
      Right((q: Q) => p1((p, q)))
    case Right(np) =>
      Left(np)
  }
}

defined function rdfNotAnd

In [98]:
/*def ex46 [P,Q,R,S,T,U,W](EM: ExcludedMiddle)(p1: ((P,Not[Q]),R)=>S, p2: T, p3: ((S,T))=>U, p4: P=>(U=>Not[W]), p5:W): (P,R)=>Q = {
    (p,r)=> val uInw = p4(p)
            val wInu = rcontr1_2(uInw) //w=>Not[u] cuando tenga ¬¬w=w
            val nu = wInu(p5)
            val n_sAt = modusTollens(p3)(nu) //Not[(S,T)]
            val e_nsnt = rdfNotAnd(EM)(n_sAt)
            val p14 = ria2_NOT(e_nsnt,p2) //ns
            //val p15 = modusTollens(p1)(p14)
            //val p16 = 
            
            
}    */

In [98]:
//ejemplo knights and knaves 
//(a → (b ∧  ¬c)) ∧ ((b ∧  ¬c) → a) 

def exKK[A, B, C](a: A, t:((B, Not[C]))): (A => ((B, Not[C])), ((B, Not[C])) => A) = {
  (a => t, t => a)
}

defined function exKK